# UK Power Networks Power Cuts - API Exploration

**Goal:** Fetch and explore live power outage data from UK Power Network. 
## API Endpoint <br>  
**CKAN DataStore API:**  https://spenergynetworks.opendatasoft.com/api/explore/v2.1/catalog/datasets/distribution-network-live-outages/records

## 0. Setup And Imports

In [25]:
import os
import json
import requests
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables
load_dotenv()
UKPN_API_KEY = os.getenv('UKPN_API_KEY')

# Display settings for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

## 1. API Configuration

In [26]:
# API Configuration
BASE_URL = "https://ukpowernetworks.opendatasoft.com/api/explore/v2.1"
DATASET_ID = "ukpn-live-faults"
API_ENDPOINT = f"{BASE_URL}/catalog/datasets/{DATASET_ID}/records"

headers = {"Authorization": f"Apikey {UKPN_API_KEY}"}
params = {"limit": 10, "timezone": "Europe/London"}

## 2. Dataset Metadata

In [27]:
# Metadata was defined on the page where the dataset is hosted
print("DATASET: UK Power Networks - Live Faults")
print("=" * 80)
print("Dataset ID:", DATASET_ID)
print("Coverage: EPN, LPN, SPN (London, Eastern, South Eastern)")
print("Privacy: Incidents affecting ≤5 customers omitted\n")

DATASET: UK Power Networks - Live Faults
Dataset ID: ukpn-live-faults
Coverage: EPN, LPN, SPN (London, Eastern, South Eastern)
Privacy: Incidents affecting ≤5 customers omitted



## 3. Fetch Data

In [28]:
response = requests.get(API_ENDPOINT, headers=headers, params=params)

if response.status_code == 200:
    data = response.json()
    total_count = data.get('total_count', 0)

    # Convert to DataFrame - try different structures
    results = data.get('results', [])
    try:
        df = pd.DataFrame([r['fields'] for r in results])
    except KeyError:
        try:
            df = pd.DataFrame([r['record']['fields'] for r in results])
        except KeyError:
            df = pd.DataFrame(results)

    # Display Schema Overview
    print("=" * 80)
    print("SCHEMA OVERVIEW")
    print("=" * 80)
    print(f"Total Records Available: {total_count}")
    print(f"Records Retrieved: {len(df)}")
    print(f"Columns: {len(df.columns)}\n")

    # Display Column Datatypes
    print("-" * 80)
    print("COLUMN DATATYPES")
    print("-" * 80)
    display(df.dtypes)

    # Display Sample Data
    print("\n" + "=" * 80)
    print("SAMPLE DATA (First 5 Rows)")
    print("=" * 80)
    display(df.head(2))

    # Key Columns Summary
    print("\n" + "-" * 80)
    print("KEY COLUMNS:")
    print("-" * 80)
    print("✓ incidentreference - Unique incident ID")
    print("✓ powercuttype - Planned/Unplanned/Restored")
    print("✓ nocustomeraffected - Number of customers affected")
    print("✓ postcodesaffected - Affected postcode sectors")
    print("✓ estimatedrestorationdate - Estimated restoration time")
    print("✓ geopoint - {'lon': x, 'lat': y} geographic coordinates")
    print("✓ operatingzone - UK Power Networks operational zone")
    print("=" * 80)

else:
    print(f"✗ Error {response.status_code}: {response.text}")

SCHEMA OVERVIEW
Total Records Available: 204
Records Retrieved: 10
Columns: 29

--------------------------------------------------------------------------------
COLUMN DATATYPES
--------------------------------------------------------------------------------


incidentreference                              object
powercuttype                                   object
creationdatetime                               object
nocallsreported                                 int64
incidentscount                                  int64
nocustomeraffected                              int64
postcodesaffected                              object
restoredincidents                              object
unplannedincidents                             object
plannedincidents                               object
incidenttypetbcestimatedfriendlydescription    object
incidentdescription                            object
fullpostcodedata                               object
incidentcategorycustomerfriendlydescription    object
incidentcategory                               object
incidenttypename                               object
incidenttype                                    int64
incidentpriority                                int64
statusid                    


SAMPLE DATA (First 5 Rows)


,incidentreference,powercuttype,creationdatetime,nocallsreported,incidentscount,nocustomeraffected,postcodesaffected,restoredincidents,unplannedincidents,plannedincidents,incidenttypetbcestimatedfriendlydescription,incidentdescription,fullpostcodedata,incidentcategorycustomerfriendlydescription,incidentcategory,incidenttypename,incidenttype,incidentpriority,statusid,restoreddatetime,planneddate,receiveddate,noplannedcustomers,plannedincidentreason,message,mainmessage,geopoint,estimatedrestorationdate,operatingzone
0,INCD-326675-U,Planned,2025-10-23T13:45:33,22,0,62,GU24 0,None,None,None,17 Nov 16:30 - 17:30,None,GU240PZ;GU240QD;GU240QG;GU240QB;GU240QE;GU240BL;GU240QF;GU240QA;GU240JB,DYNAMIC ETR 16:30 - 17:30,28,Planned,3,12,1,None,2025-11-17T09:00:00,2025-11-17T09:18:27,62,DYNAMIC ETR 16:30 - 17:30,None,The electricity supply to your premises may currently be affected by planned work. You should have received a letter explaining this. We hope to be able to restore your supplies by 17-NOV-2025 17:00. Please accept our sincere apologies for any inconvenience this is causing.,"{'lon': -0.6562, 'lat': 51.2908}",2025-11-17T17:00:00,Kingston Zone
1,INCD-105128-C,Planned,2025-10-27T10:28:40,0,0,0,NR12 8,None,None,None,17 Nov 11:00 - 12:00,None,NR128UJ;NR128UN,We're carrying out planned work in your area. For our engineers to carry it out safely they need to turn the power off. We're doing this work as it's essential to provide reliable electricity supplies to your area. We're sorry for any inconvenience caused and thank you for your patience.,28,Planned,3,12,1,None,2025-11-17T09:00:00,2025-11-17T09:00:00,0,We're carrying out planned work in your area. For our engineers to carry it out safely they need to turn the power off. We're doing this work as it's essential to provide reliable electricity supplies to your area. We're sorry for any inconvenience caused and thank you for your patience.,None,The electricity supply to your premises may currently be affected by planned work. You should have received a letter explaining this. We hope to be able to restore your supplies by 17-NOV-2025 11:30. Please accept our sincere apologies for any inconvenience this is causing.,None,2025-11-17T11:30:00,NORWICH



--------------------------------------------------------------------------------
KEY COLUMNS:
--------------------------------------------------------------------------------
✓ incidentreference - Unique incident ID
✓ powercuttype - Planned/Unplanned/Restored
✓ nocustomeraffected - Number of customers affected
✓ postcodesaffected - Affected postcode sectors
✓ estimatedrestorationdate - Estimated restoration time
✓ geopoint - {'lon': x, 'lat': y} geographic coordinates
✓ operatingzone - UK Power Networks operational zone
